In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

### YOLOv8을 사용한 얼굴 부위 분류 모델 학습
- 학습결과: 훈련 데이터셋이 모두 같은 장소에서 찍어 비슷하여 모델이 너무 훈련 데이터셋에 맞춰져있는 것 같아 새로운 이미지를 테스트 해봤을 때 높은 정확율을 보일 때도 있지만 오분류가 일어나는 경우도 있습니다.

In [ ]:
!pip install ultralytics

In [ ]:
import os
import json
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
from PIL import Image
import shutil
import ast
from IPython.display import display, Javascript
import random
from itertools import product

In [ ]:
# 런타임 오류 방지
def keep_alive():
    display(Javascript('''
        function ClickConnect(){
            console.log("Clicking connect button");
            document.querySelector("colab-connect-button").click()
        }
        setInterval(ClickConnect, 60000)
    '''))

In [ ]:
# YAML 파일 생성 함수
def create_yolo_yaml(base_path, save_dir):
    yaml_content = f"""
path: {base_path}
train: Training/images
val: Validation/images
nc: 9
names: ['face', 'forehead', 'glabellus', 'l_perocular', 'r_perocular', 'l_cheek', 'r_cheek', 'lip', 'chin']

train_labels: Training/labels
val_labels: Validation/labels
"""
    yaml_path = os.path.join(save_dir, 'dataset.yaml')
    with open(yaml_path, 'w', encoding='utf-8') as f:
        f.write(yaml_content)
    return yaml_path

In [ ]:
# 손상된 이미지와 라벨링 같이 제거
def remove_corrupted_images(folder_path):
    corrupted_images = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(folder_path, filename)
            try:
                with Image.open(file_path) as img:
                    img.verify()
            except Exception as e:
                print(f"손상된 이미지 파일 발견: {filename}")
                corrupted_images.append(filename)
                os.remove(file_path)
    return corrupted_images

In [ ]:
# yolo형식으로 데이터 전처리 함수
def prepare_yolo_data(df, base_path):
    train_image_path = os.path.join(base_path, 'Training', 'images')
    val_image_path = os.path.join(base_path, 'Validation', 'images')
    train_label_path = os.path.join(base_path, 'Training', 'labels')
    val_label_path = os.path.join(base_path, 'Validation', 'labels')

    # 모든 필요한 폴더가 이미 존재하는지 확인
    if all(os.path.exists(path) for path in [train_image_path, val_image_path, train_label_path, val_label_path]):
        print("기존 데이터 폴더를 사용합니다.")
        return train_image_path, val_image_path, train_label_path, val_label_path

    # 폴더가 없는 경우 새로 생성
    os.makedirs(train_image_path, exist_ok=True)
    os.makedirs(val_image_path, exist_ok=True)
    os.makedirs(train_label_path, exist_ok=True)
    os.makedirs(val_label_path, exist_ok=True)

    print("Training 데이터에서 손상된 이미지 제거 중...")
    remove_corrupted_images(train_image_path)
    print("Validation 데이터에서 손상된 이미지 제거 중...")
    remove_corrupted_images(val_image_path)

    processed_count = 0
    for index, row in df.iterrows():
        try:
            info = row['info']
            images = row['images']

            filename = info['filename']
            facepart = images['facepart']
            bbox = images['bbox']
            width = images['width']
            height = images['height']

            src_img_path = os.path.join(base_path, row['split'], '01.원천데이터', filename)
            dst_img_path = os.path.join(train_image_path if 'Training' in row['split'] else val_image_path, filename)

            if not os.path.exists(src_img_path):
                print(f"{index}번 행을 건너뜁니다. 이미지 파일이 존재하지 않습니다: {filename}")
                continue

            # 이미지 파일 복사
            shutil.copy2(src_img_path, dst_img_path)

            if not all([filename, facepart is not None, width, height, bbox]):
                print(f"{index}번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: {filename}")
                continue

            if (bbox == ['None', 'None', 'None', 'None']) or not all(isinstance(b, (int, float)) and b is not None for b in bbox):
                print(f"{index}번 행을 건너뜁니다. 유효하지 않은 bbox 값입니다. 파일: {filename}")
                continue

            bbox = [float(b) for b in bbox]
            width = float(width)
            height = float(height)

            x_center = (bbox[0] + bbox[2]) / 2 / width
            y_center = (bbox[1] + bbox[3]) / 2 / height
            bbox_width = (bbox[2] - bbox[0]) / width
            bbox_height = (bbox[3] - bbox[1]) / height

            yolo_format = f"{facepart} {x_center} {y_center} {bbox_width} {bbox_height}\n"

            label_path = train_label_path if 'Training' in row['split'] else val_label_path
            label_filename = os.path.splitext(filename)[0] + '.txt'

            with open(os.path.join(label_path, label_filename), 'a') as f:
                f.write(yolo_format)

            processed_count += 1

        except Exception as e:
            print(f"{index}번 행 처리 중 오류 발생: {str(e)}")
            print(f"행 데이터: {row}")

    print(f"총 {processed_count}개의 이미지를 처리했습니다.")
    return train_image_path, val_image_path, train_label_path, val_label_path

In [ ]:
def train_yolo_model(df, base_path, save_dir):
    train_image_path, val_image_path, train_label_path, val_label_path = prepare_yolo_data(df, base_path)

    yaml_path = create_yolo_yaml(base_path, save_dir)

    # YOLO 모델 초기화 (사전 훈련된 가중치 없이)
    model = YOLO('yolov8n.yaml')

    # keep_alive 함수를 호출하여 연결 유지 시작
    keep_alive()

    # 모델 학습 시작
    results = model.train(
        data=yaml_path,
        epochs=30,
        imgsz=640,
        verbose=True,
        project=save_dir,  # 결과를 저장할 디렉토리 지정
        name='yolo_facepart_model'  # 결과 폴더의 이름 지정
    )
    return model, results

In [ ]:
# 메인 실행 부분
if __name__ == "__main__":
    base_path = os.path.abspath('/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터')
    save_dir = '/gdrive/MyDrive/Final project/1_Red/5_분석모델링/YOLOv8'

    # 저장한 데이터 프레임 불러오기
    df = pd.read_csv(base_path + '/json to df.csv')

    # csv로 불러오니 딕셔너리가 문자열로 다 바뀌어서 나와 딕셔너리로 변환
    for col in ['info', 'images', 'annotations', 'equipment']:
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if x else None)

    # YOLO 모델 학습
    trained_model, training_results = train_yolo_model(df, base_path, save_dir)

    # 학습이 완료된 후 keep_alive 함수를 한 번 더 호출
    keep_alive()

    if trained_model:
        print("모델 학습이 완료되었습니다.")
        print("학습 결과:", training_results)
        print(f"학습된 모델이 '{save_dir}/yolo_facepart_model'에 저장되었습니다.")
    else:
        print("모델 학습에 실패했습니다.")

### YOLOv8을 사용한 얼굴 부위 분류 모델 학습(과적합 및 증강 이용)
- epoch = 30
    - 위 코드에서 epoch 30으로 좋은 성능을 내어서 epoch는 동일하게 사용<br><br>
- weight_decay=0.0005
    - 가중치 감소를 이용해 모델 파라미터가 너무 커지는 것을 방지하여 모델이 너무 복잡해지는 것을 방지<br><br>
- dropout=0.2
    - 훈련 중 일부 뉴런을 랜덤하게 비활성화하여 과적합을 줄이기<br><br>
- augment=True, mosaic=0.5, mixup=0.2
    - 모자이크와 믹스업을 이용하여 이미지를 조합해 새로운 훈련 데이터를 생성하여 사용<br><br>
- cos_lr=True, lr0=0.01, lrf=0.01
    - 모델의 학습 속도를 조절하기 위해 사용, 처음에는 빠르게 학습하다가 점점 속도를 줄여 세밀한 조정을 하여 학습 시간을 조절<br><br>
- patience=10
    - 모델의 성능이 10 epoch 동안 개선되지 않으면 조기 종료

In [ ]:
# 런타임 오류 방지
def keep_alive():
    display(Javascript('''
        function ClickConnect(){
            console.log("Clicking connect button");
            document.querySelector("colab-connect-button").click()
        }
        setInterval(ClickConnect, 60000)
    '''))

In [ ]:
# YAML 파일 생성 함수
def create_yolo_yaml(base_path, save_dir):
    yaml_content = f"""
path: {base_path}
train: Training/images
val: Validation/images
nc: 9
names: ['face', 'forehead', 'glabellus', 'l_perocular', 'r_perocular', 'l_cheek', 'r_cheek', 'lip', 'chin']

train_labels: Training/labels
val_labels: Validation/labels
"""
    yaml_path = os.path.join(save_dir, 'dataset.yaml')
    with open(yaml_path, 'w', encoding='utf-8') as f:
        f.write(yaml_content)
    return yaml_path

In [ ]:
# 손상된 이미지와 라벨링 같이 제거
def remove_corrupted_images(folder_path):
    corrupted_images = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            file_path = os.path.join(folder_path, filename)
            try:
                with Image.open(file_path) as img:
                    img.verify()
            except Exception as e:
                print(f"손상된 이미지 파일 발견: {filename}")
                corrupted_images.append(filename)
                os.remove(file_path)
    return corrupted_images

In [ ]:
# 이미지와 바운딩 박스 증강 함수 / 밝기 조정, 대비 조정, 회전, 플립 등 랜덤한 변환 적용
def augment_image_and_bbox(image, bbox):
    h, w = image.shape[:2]
    x, y, bw, bh = bbox

    # 랜덤 밝기 조정
    brightness = random.uniform(0.8, 1.2)
    image = cv2.convertScaleAbs(image, alpha=brightness, beta=0)

    # 랜덤 대비 조정
    contrast = random.uniform(0.8, 1.2)
    mean = np.mean(image)
    image = np.clip((image - mean) * contrast + mean, 0, 255).astype(np.uint8)

    # 랜덤 회전
    angle = random.uniform(-15, 15)
    M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1)
    image = cv2.warpAffine(image, M, (w, h))

    # 바운딩 박스 회전
    bbox_points = np.array([[x, y], [x+bw, y], [x, y+bh], [x+bw, y+bh]], dtype=np.float32)
    bbox_points = np.hstack((bbox_points, np.ones((4,1))))
    rotated_bbox = M.dot(bbox_points.T).T
    rotated_bbox = rotated_bbox[:,:2]
    new_x = np.min(rotated_bbox[:,0])
    new_y = np.min(rotated_bbox[:,1])
    new_bw = np.max(rotated_bbox[:,0]) - new_x
    new_bh = np.max(rotated_bbox[:,1]) - new_y

    # 랜덤 플립
    if random.random() > 0.5:
        image = cv2.flip(image, 1)
        new_x = w - new_x - new_bw

    # 바운딩 박스가 이미지 범위를 벗어나지 않도록 조정
    new_x = max(0, min(new_x, w - 1))
    new_y = max(0, min(new_y, h - 1))
    new_bw = min(new_bw, w - new_x)
    new_bh = min(new_bh, h - new_y)

    return image, [new_x, new_y, new_bw, new_bh]

In [ ]:
# YOLO 형식으로 데이터 전처리 함수
def prepare_yolo_data(df, base_path, augment=True):
    train_image_path = os.path.join(base_path, 'Training', 'images')
    val_image_path = os.path.join(base_path, 'Validation', 'images')
    train_label_path = os.path.join(base_path, 'Training', 'labels')
    val_label_path = os.path.join(base_path, 'Validation', 'labels')

    # 모든 필요한 폴더가 이미 존재하는지 확인
    if all(os.path.exists(path) for path in [train_image_path, val_image_path, train_label_path, val_label_path]):
        print("기존 데이터 폴더를 사용합니다.")
        return train_image_path, val_image_path, train_label_path, val_label_path

    # 폴더가 없는 경우 새로 생성
    os.makedirs(train_image_path, exist_ok=True)
    os.makedirs(val_image_path, exist_ok=True)
    os.makedirs(train_label_path, exist_ok=True)
    os.makedirs(val_label_path, exist_ok=True)

    print("Training 데이터에서 손상된 이미지 제거 중...")
    remove_corrupted_images(train_image_path)
    print("Validation 데이터에서 손상된 이미지 제거 중...")
    remove_corrupted_images(val_image_path)

    processed_count = 0
    for index, row in df.iterrows():
        try:
            info = row['info']
            images = row['images']

            filename = info['filename']
            facepart = images['facepart']
            bbox = images['bbox']
            width = images['width']
            height = images['height']

            src_img_path = os.path.join(base_path, row['split'], '01.원천데이터', filename)
            dst_img_path = os.path.join(train_image_path if 'Training' in row['split'] else val_image_path, filename)

            if not os.path.exists(src_img_path):
                print(f"{index}번 행을 건너뜁니다. 이미지 파일이 존재하지 않습니다: {filename}")
                continue

            # 이미지 파일 복사
            shutil.copy2(src_img_path, dst_img_path)

            if not all([filename, facepart is not None, width, height, bbox]):
                print(f"{index}번 행을 건너뜁니다. 파일에 대한 필수 정보가 누락되었습니다: {filename}")
                continue

            if (bbox == ['None', 'None', 'None', 'None']) or not all(isinstance(b, (int, float)) and b is not None for b in bbox):
                print(f"{index}번 행을 건너뜁니다. 유효하지 않은 bbox 값입니다. 파일: {filename}")
                continue

            bbox = [float(b) for b in bbox]
            width = float(width)
            height = float(height)

            # YOLO 형식으로 변환
            x_center = (bbox[0] + bbox[2] / 2) / width
            y_center = (bbox[1] + bbox[3] / 2) / height
            bbox_width = bbox[2] / width
            bbox_height = bbox[3] / height

            yolo_format = f"{facepart} {x_center} {y_center} {bbox_width} {bbox_height}\n"

            label_path = train_label_path if 'Training' in row['split'] else val_label_path
            label_filename = os.path.splitext(filename)[0] + '.txt'

            with open(os.path.join(label_path, label_filename), 'a') as f:
                f.write(yolo_format)

            # 데이터 증강 (훈련 데이터에만 적용)
            if augment and 'Training' in row['split']:
                img = cv2.imread(dst_img_path)
                for i in range(2):  # 각 이미지에 대해 2개의 증강된 이미지 생성
                    aug_img, aug_bbox = augment_image_and_bbox(img, bbox)
                    aug_filename = f"{os.path.splitext(filename)[0]}_aug_{i}.jpg"
                    cv2.imwrite(os.path.join(train_image_path, aug_filename), aug_img)

                    # 증강된 이미지의 바운딩 박스를 YOLO 형식으로 변환
                    aug_x_center = (aug_bbox[0] + aug_bbox[2] / 2) / width
                    aug_y_center = (aug_bbox[1] + aug_bbox[3] / 2) / height
                    aug_bbox_width = aug_bbox[2] / width
                    aug_bbox_height = aug_bbox[3] / height

                    aug_yolo_format = f"{facepart} {aug_x_center} {aug_y_center} {aug_bbox_width} {aug_bbox_height}\n"

                    with open(os.path.join(train_label_path, f"{os.path.splitext(aug_filename)[0]}.txt"), 'w') as f:
                        f.write(aug_yolo_format)

            processed_count += 1

        except Exception as e:
            print(f"{index}번 행 처리 중 오류 발생: {str(e)}")
            print(f"행 데이터: {row}")

    print(f"총 {processed_count}개의 이미지를 처리했습니다.")
    return train_image_path, val_image_path, train_label_path, val_label_path

In [ ]:
# YOLO 모델 학습 함수
def train_yolo_model(df, base_path, save_dir, hyper_params):
    train_image_path, val_image_path, train_label_path, val_label_path = prepare_yolo_data(df, base_path)

    yaml_path = create_yolo_yaml(base_path, save_dir)

    # YOLO 모델 초기화 (사전 훈련된 가중치 사용)
    model = YOLO('yolov8n.pt')

    # keep_alive 함수를 호출하여 연결 유지 시작
    keep_alive()

    # 모델 학습 시작
    results = model.train(
        data=yaml_path,
        epochs=hyper_params['epochs'],
        imgsz=640,
        batch=16,
        workers=8,
        verbose=True,
        project=save_dir,
        name=f'yolo_facepart_model_{hyper_params["epochs"]}_{hyper_params["dropout"]}_{hyper_params["mixup"]}',
        patience=10,
        dropout=hyper_params['dropout'],
        weight_decay=0.0005,
        mosaic=0.5,
        mixup=hyper_params['mixup'],
        augment=True,
        cos_lr=True,
        lr0=0.01,
        lrf=0.01,
        save_period=1  # 매 에포크마다 체크포인트 저장
    )
    return model, results

In [ ]:
# 하이퍼파라미터 랜덤 서치 함수
def random_search(df, base_path, save_dir, num_trials=5):
    hyper_param_space = {
        'epochs': [20, 25, 30],
        'dropout': [0.3, 0.4, 0.5],
        'mixup': [0.2, 0.3, 0.4]
    }

    best_performance = float('-inf')
    best_params = None
    all_results = []

    for _ in range(num_trials):
        hyper_params = {k: random.choice(v) for k, v in hyper_param_space.items()}

        print(f"현재 시도 중인 하이퍼파라미터: {hyper_params}")

        trained_model, training_results = train_yolo_model(df, base_path, save_dir, hyper_params)

        # 성능 메트릭을 가져옵니다
        performance = training_results.results_dict['metrics/mAP50-95(B)']

        all_results.append((hyper_params, performance))

        if performance > best_performance:
            best_performance = performance
            best_params = hyper_params

        print(f"현재 하이퍼파라미터의 성능: {performance}")
        print(f"현재까지의 최고 성능: {best_performance} (파라미터: {best_params})")

        # keep_alive 함수를 한 번 더 호출하여 연결 유지
        keep_alive()

    return all_results, best_params, best_performance

In [ ]:
# 메인 실행 부분
if __name__ == "__main__":
    base_path = os.path.abspath('/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터')
    save_dir = '/gdrive/MyDrive/Final project/1_Red/5_분석모델링/YOLOv8'

    # 저장한 데이터 프레임 불러오기
    df = pd.read_csv(base_path + '/json to df.csv')

    # csv로 불러오니 딕셔너리가 문자열로 다 바뀌어서 나와 딕셔너리로 변환
    for col in ['info', 'images', 'annotations', 'equipment']:
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if x else None)

    # 랜덤 서치 수행
    all_results, best_params, best_performance = random_search(df, base_path, save_dir, num_trials=5)

    # 결과 출력
    print("\n모든 시도의 결과:")
    for params, performance in all_results:
        print(f"하이퍼파라미터: {params}, 성능: {performance}")

    print(f"\n최고 성능: {best_performance}")
    print(f"최고 성능의 하이퍼파라미터: {best_params}")

    # 학습이 완료된 후 keep_alive 함수를 한 번 더 호출
    keep_alive()

In [ ]:
# 중단됐을 경우 이어시 학습
def resume_yolo_training(df, base_path, save_dir, checkpoint_path):
    train_image_path, val_image_path, train_label_path, val_label_path = prepare_yolo_data(df, base_path)
    yaml_path = create_yolo_yaml(base_path, save_dir)

    # 중단된 지점의 체크포인트로부터 모델 로드
    model = YOLO(checkpoint_path)

    # keep_alive 함수를 호출하여 연결 유지 시작
    keep_alive()

    # 모델 학습 재개
    results = model.train(
        data=yaml_path,
        epochs=30,  # 총 원하는 에폭 수 설정
        imgsz=640,
        batch=16,
        workers=8,
        verbose=True,
        project=save_dir,
        name='yolo_facepart_model_overfitting',  # 기존 폴더명 유지
        patience=10,
        dropout=0.2,
        weight_decay=0.0005,
        mosaic=0.5,
        mixup=0.2,
        augment=True,
        cos_lr=True,
        lr0=0.01,
        lrf=0.01,
        resume=True  # 학습 재개 플래그
    )
    return model, results

# 학습 재개 실행
base_path = os.path.abspath('/gdrive/MyDrive/Final project/1_Red/3_데이터수집_저장/0_데이터수집폴더/피부 데이터')
save_dir = '/gdrive/MyDrive/Final project/1_Red/5_분석모델링/YOLOv8'

# 저장한 데이터 프레임 불러오기
df = pd.read_csv(base_path + '/json to df.csv')

# csv로 불러오니 딕셔너리가 문자열로 다 바뀌어서 나와 딕셔너리로 변환
for col in ['info', 'images', 'annotations', 'equipment']:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if x else None)
checkpoint_path = '/gdrive/MyDrive/Final project/1_Red/5_분석모델링/YOLOv8/yolo_facepart_model_overfitting/weights/last.pt'
model, results = resume_yolo_training(df, base_path, save_dir, checkpoint_path)

### 테스트

In [ ]:
import os
import cv2
import numpy as np
from PIL import Image
from ultralytics import YOLO

def resize_image(img, target_size):
    h, w = img.shape[:2]
    ratio = min(target_size/h, target_size/w)
    new_size = (int(w*ratio), int(h*ratio))
    resized = cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)

    delta_w = target_size - new_size[0]
    delta_h = target_size - new_size[1]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    padded = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0,0,0])
    return padded, (top, left), ratio

def predict_and_visualize(model, image_path, output_path, conf=0.25, iou=0.45, max_det=300):
    results = model.predict(image_path, conf=conf, iou=iou, max_det=max_det)

    for r in results:
        im_array = r.plot()  # plot a BGR numpy array of predictions
        im = Image.fromarray(im_array[..., ::-1])  # RGB PIL image
        im.save(output_path)  # save image

    return results

def test_yolo_model(model_path, test_images_folder, output_folder, target_size=640):
    model = YOLO(model_path)
    os.makedirs(output_folder, exist_ok=True)

    for img_name in os.listdir(test_images_folder):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(test_images_folder, img_name)
            img_output_folder = os.path.join(output_folder, os.path.splitext(img_name)[0])
            os.makedirs(img_output_folder, exist_ok=True)

            img = cv2.imread(img_path)
            if img is None:
                print(f"이미지를 로드할 수 없습니다: {img_path}")
                continue

            resized_img, (pad_top, pad_left), resize_ratio = resize_image(img, target_size)

            # predict_and_visualize 함수 사용
            annotated_output_path = os.path.join(img_output_folder, f"annotated_{img_name}")
            results = predict_and_visualize(model, resized_img, annotated_output_path)

            for r in results:
                boxes = r.boxes
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0]

                    x1 = max(0, int((x1 - pad_left) / resize_ratio))
                    y1 = max(0, int((y1 - pad_top) / resize_ratio))
                    x2 = min(img.shape[1], int((x2 - pad_left) / resize_ratio))
                    y2 = min(img.shape[0], int((y2 - pad_top) / resize_ratio))

                    class_name = model.names[int(box.cls)]

                    if x1 < x2 and y1 < y2:
                        cropped_img = img[y1:y2, x1:x2]

                        if cropped_img.size > 0:
                            cv2.imwrite(os.path.join(img_output_folder, f"{class_name}_{img_name}"), cropped_img)
                        else:
                            print(f"Warning: 빈 이미지가 생성되었습니다. {img_name}의 {class_name} 부분")
                    else:
                        print(f"Warning: 유효하지 않은 바운딩 박스입니다. {img_name}의 {class_name} 부분")

            print(f"처리 완료: {img_name}")

    print("모든 테스트 이미지 처리 완료")

# 함수 실행
if __name__ == "__main__":
    model_path = '/content/runs/detect/train/weights/best.pt'  # best.pt 파일 경로
    test_images_folder = '/gdrive/MyDrive/Final project/test'  # 테스트 이미지가 있는 폴더 경로
    output_folder = '/gdrive/MyDrive/Final project/test_output9'  # 결과를 저장할 폴더 경로
    target_size = 640  # YOLO 모델이 학습된 이미지 크기

    test_yolo_model(model_path, test_images_folder, output_folder, target_size)

### 첫 번째 모델 테스트 코드

In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO

def resize_image(img, target_size):
    # 원본 이미지의 비율을 유지하면서 리사이즈
    h, w = img.shape[:2]
    ratio = min(target_size/h, target_size/w)
    new_size = (int(w*ratio), int(h*ratio))
    resized = cv2.resize(img, new_size, interpolation=cv2.INTER_AREA)

    # 타겟 사이즈에 맞게 패딩
    delta_w = target_size - new_size[0]
    delta_h = target_size - new_size[1]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    padded = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0,0,0])
    return padded, (top, left), ratio

def test_yolo_model(model_path, test_images_folder, output_folder, target_size=640):
    # YOLO 모델 로드
    model = YOLO(model_path)

    # 출력 폴더 생성
    os.makedirs(output_folder, exist_ok=True)

    # 테스트 이미지 처리
    for img_name in os.listdir(test_images_folder):
        if img_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(test_images_folder, img_name)

            # 이미지별 출력 폴더 생성
            img_output_folder = os.path.join(output_folder, os.path.splitext(img_name)[0])
            os.makedirs(img_output_folder, exist_ok=True)

            # 이미지 로드
            img = cv2.imread(img_path)
            if img is None:
                print(f"이미지를 로드할 수 없습니다: {img_path}")
                continue

            # 이미지 리사이즈
            resized_img, (pad_top, pad_left), resize_ratio = resize_image(img, target_size)

            # 예측 수행
            results = model(resized_img)

            # 원본 이미지에 바운딩 박스 그리기
            annotated_img = results[0].plot()

            # 바운딩 박스가 그려진 전체 이미지 저장
            cv2.imwrite(os.path.join(img_output_folder, f"annotated_{img_name}"), annotated_img)

            # 각 얼굴 부위별로 이미지 자르기
            for r in results:
                boxes = r.boxes
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0]

                    # 패딩과 리사이즈 비율을 고려하여 원본 이미지에서의 좌표 계산
                    x1 = max(0, int((x1 - pad_left) / resize_ratio))
                    y1 = max(0, int((y1 - pad_top) / resize_ratio))
                    x2 = min(img.shape[1], int((x2 - pad_left) / resize_ratio))
                    y2 = min(img.shape[0], int((y2 - pad_top) / resize_ratio))

                    # 클래스 이름 가져오기
                    class_name = model.names[int(box.cls)]

                    # 이미지 자르기
                    if x1 < x2 and y1 < y2:  # 유효한 박스인지 확인
                        cropped_img = img[y1:y2, x1:x2]

                        # 잘린 이미지가 비어있지 않은지 확인
                        if cropped_img.size > 0:
                            # 잘린 이미지 저장
                            cv2.imwrite(os.path.join(img_output_folder, f"{class_name}_{img_name}"), cropped_img)
                        else:
                            print(f"Warning: 빈 이미지가 생성되었습니다. {img_name}의 {class_name} 부분")
                    else:
                        print(f"Warning: 유효하지 않은 바운딩 박스입니다. {img_name}의 {class_name} 부분")

            print(f"처리 완료: {img_name}")

    print("모든 테스트 이미지 처리 완료")

# 함수 실행
if __name__ == "__main__":
    model_path = '/gdrive/MyDrive/Final project/1_Red/5_분석모델링/YOLOv8/yolo_facepart_model_25_0.4_0.2/weights/best.pt'  # best.pt 파일 경로
    test_images_folder = '/gdrive/MyDrive/Final project/test'  # 테스트 이미지가 있는 폴더 경로
    output_folder = '/gdrive/MyDrive/Final project/test_output_25_0.4_0.2'  # 결과를 저장할 폴더 경로
    target_size = 640  # YOLO 모델이 학습된 이미지 크기

    test_yolo_model(model_path, test_images_folder, output_folder, target_size)